# Ejercicio EvCont4 (a) - Compresión LZZ7

* Óscar Sementé Solà
* Abdelkarim Azzouguagh Ouniri
* Rodrigo Cabezas Quirós

In [38]:
import re
[(m.start(0), m.end(0)) for m in re.finditer("1101", msg)]
#re.findall(r"1101", msg)

[(0, 4), (15, 19)]

In [39]:
import math, re

class LZZ7Compressor:
    
    def __init__(self, win, wsld):
        self.win = win
        self.wsld = wsld
        self.win_b = int(math.log2(win))
        self.wsld_b = int(math.log2(wsld))
    
    def __is_power_of_two(v):
        return math.log2(v).is_integer()
    
    def __ld_to_bin(self, l, d):
        b_win = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(self.win_b)]))
        b_wsld = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(self.wsld_b)]))
        return b_win(l) + b_wsld(d)
        
    def compress(m):
        if self.win > self.wsld or len(m) < self.win + self.wsld or not self.__is_power_of_two(self.win) \
            or not self.__is_power_of_two(self.wsld):
            return -1
        l1, l2, l3 = 0, self.wsld, self.win
        while l3 <= len(m):
            occ = [(x.start(0), x.end(0)) for x in re.finditer("1101", msg)]
            
    

SyntaxError: unexpected EOF while parsing (<ipython-input-39-6f90ecf5e771>, line 26)

In [33]:
msg = "11011100101001111010100010001"
comp = LZZ7Compressor(4, 8)

#### [Apartado 1] Comprobad que el programa comprime y descomprime correctamente una cadena de 25 bits aleatorios con Mdes = 8 y Ment = 4. Ayuda: Podéis generar bits aleatorios con Math.round(Math.random()).

#### [Apartado 2] Utilizad el programa anterior para determinar si es posible, ajustando los valores de Mdes y Ment, conseguir comprimir datos aleatorios mediante LZ77 (es decir, que la cadena de datos originales sea más larga que la cadena comprimida). ¿Por qué? ¿Cuál es la máxima compresión que lográis? ¿Con qué valores? (Ayuda: utilizad una cadena de datos de entrada de, por lo menos, 10000 bits aleatorios. Ajustad Mdes y Ment entre 2 y 2048).